# Comparison Between TreeValue and Tianshou Batch

In this section, we will take a look at the feature and performance of the [Tianshou Batch](https://github.com/thu-ml/tianshou) library, which is developed by Tsinghua Machine Learning Group.

Before starting the comparison, let us define some thing.

In [1]:
import torch 

_TREE_DATA_1 = {'a': 1, 'b': 2, 'x': {'c': 3, 'd': 4}}
_TREE_DATA_2 = {
    'a': torch.randn(2, 3), 
    'x': {
        'c': torch.randn(3, 4)
    },
}
_TREE_DATA_3 = {
    'obs': torch.randn(4, 84, 84),
    'action': torch.randint(0, 6, size=(1,)),
    'reward': torch.rand(1),
}

/opt/hostedtoolcache/Python/3.8.16/x64/lib/python3.8/site-packages/tqdm/auto.py:22: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


## Read and Write Operation

Reading and writing are the two most common operations in the tree data structure based on the data model (TreeValue and Tianshou Batch both belong to this type), so this section will compare the reading and writing performance of these two libraries.

### TreeValue's Get and Set

In [2]:
from treevalue import FastTreeValue

t = FastTreeValue(_TREE_DATA_2)

In [3]:
t

<FastTreeValue 0x7f9e38f17a90>
├── 'a' --> tensor([[ 1.6330,  1.1883,  0.1030],
│                   [-0.1685,  1.1185, -0.6797]])
└── 'x' --> <FastTreeValue 0x7f9e38f8be50>
    └── 'c' --> tensor([[ 0.1707,  0.0238,  0.1529,  0.9907],
                        [-1.9703,  1.0273, -0.1173, -0.6577],
                        [ 0.6435,  0.9898, -1.5166, -0.4396]])

In [4]:
t.a

tensor([[ 1.6330,  1.1883,  0.1030],
        [-0.1685,  1.1185, -0.6797]])

In [5]:
%timeit t.a

61.5 ns ± 0.0347 ns per loop (mean ± std. dev. of 7 runs, 10,000,000 loops each)


In [6]:
new_value = torch.randn(2, 3)
t.a = new_value

t

<FastTreeValue 0x7f9e38f17a90>
├── 'a' --> tensor([[ 0.5375, -1.1565,  0.9579],
│                   [ 0.4841,  1.7221, -0.8098]])
└── 'x' --> <FastTreeValue 0x7f9e38f8be50>
    └── 'c' --> tensor([[ 0.1707,  0.0238,  0.1529,  0.9907],
                        [-1.9703,  1.0273, -0.1173, -0.6577],
                        [ 0.6435,  0.9898, -1.5166, -0.4396]])

In [7]:
%timeit t.a = new_value

60.2 ns ± 0.215 ns per loop (mean ± std. dev. of 7 runs, 10,000,000 loops each)


### Tianshou Batch's Get and Set

In [8]:
from tianshou.data import Batch

b = Batch(**_TREE_DATA_2)

In [9]:
b

Batch(
    a: tensor([[ 1.6330,  1.1883,  0.1030],
               [-0.1685,  1.1185, -0.6797]]),
    x: Batch(
           c: tensor([[ 0.1707,  0.0238,  0.1529,  0.9907],
                      [-1.9703,  1.0273, -0.1173, -0.6577],
                      [ 0.6435,  0.9898, -1.5166, -0.4396]]),
       ),
)

In [10]:
b.a

tensor([[ 1.6330,  1.1883,  0.1030],
        [-0.1685,  1.1185, -0.6797]])

In [11]:
%timeit b.a

57.5 ns ± 0.0664 ns per loop (mean ± std. dev. of 7 runs, 10,000,000 loops each)


In [12]:
new_value = torch.randn(2, 3)
b.a = new_value

b

Batch(
    a: tensor([[ 0.3203, -2.3172, -0.1856],
               [-0.3735, -1.7583, -0.5673]]),
    x: Batch(
           c: tensor([[ 0.1707,  0.0238,  0.1529,  0.9907],
                      [-1.9703,  1.0273, -0.1173, -0.6577],
                      [ 0.6435,  0.9898, -1.5166, -0.4396]]),
       ),
)

In [13]:
%timeit b.a = new_value

478 ns ± 0.0906 ns per loop (mean ± std. dev. of 7 runs, 1,000,000 loops each)


## Initialization

### TreeValue's Initialization

In [14]:
%timeit FastTreeValue(_TREE_DATA_1)

850 ns ± 0.207 ns per loop (mean ± std. dev. of 7 runs, 1,000,000 loops each)


### Tianshou Batch's Initialization

In [15]:
%timeit Batch(**_TREE_DATA_1)

11.3 µs ± 24 ns per loop (mean ± std. dev. of 7 runs, 100,000 loops each)


## Deep Copy Operation

In [16]:
import copy

### Deep Copy of TreeValue

In [17]:
t3 = FastTreeValue(_TREE_DATA_3)
%timeit copy.deepcopy(t3)

136 µs ± 1.4 µs per loop (mean ± std. dev. of 7 runs, 1,000 loops each)


### Deep Copy of Tianshou Batch

In [18]:
b3 = Batch(**_TREE_DATA_3)
%timeit copy.deepcopy(b3)

138 µs ± 640 ns per loop (mean ± std. dev. of 7 runs, 10,000 loops each)


## Stack, Concat and Split Operation

### Performance of TreeValue

In [19]:
trees = [FastTreeValue(_TREE_DATA_2) for _ in range(8)]

In [20]:
t_stack = FastTreeValue.func(subside=True)(torch.stack)

t_stack(trees)

<FastTreeValue 0x7f9e38f1f4f0>
├── 'a' --> tensor([[[ 1.6330,  1.1883,  0.1030],
│                    [-0.1685,  1.1185, -0.6797]],
│           
│                   [[ 1.6330,  1.1883,  0.1030],
│                    [-0.1685,  1.1185, -0.6797]],
│           
│                   [[ 1.6330,  1.1883,  0.1030],
│                    [-0.1685,  1.1185, -0.6797]],
│           
│                   [[ 1.6330,  1.1883,  0.1030],
│                    [-0.1685,  1.1185, -0.6797]],
│           
│                   [[ 1.6330,  1.1883,  0.1030],
│                    [-0.1685,  1.1185, -0.6797]],
│           
│                   [[ 1.6330,  1.1883,  0.1030],
│                    [-0.1685,  1.1185, -0.6797]],
│           
│                   [[ 1.6330,  1.1883,  0.1030],
│                    [-0.1685,  1.1185, -0.6797]],
│           
│                   [[ 1.6330,  1.1883,  0.1030],
│                    [-0.1685,  1.1185, -0.6797]]])
└── 'x' --> <FastTreeValue 0x7f9e38ee2040>
    └── 'c' --> tensor([[[

In [21]:
%timeit t_stack(trees)

32.6 µs ± 35.2 ns per loop (mean ± std. dev. of 7 runs, 10,000 loops each)


In [22]:
t_cat = FastTreeValue.func(subside=True)(torch.cat)

t_cat(trees)

<FastTreeValue 0x7f9e38f1ff10>
├── 'a' --> tensor([[ 1.6330,  1.1883,  0.1030],
│                   [-0.1685,  1.1185, -0.6797],
│                   [ 1.6330,  1.1883,  0.1030],
│                   [-0.1685,  1.1185, -0.6797],
│                   [ 1.6330,  1.1883,  0.1030],
│                   [-0.1685,  1.1185, -0.6797],
│                   [ 1.6330,  1.1883,  0.1030],
│                   [-0.1685,  1.1185, -0.6797],
│                   [ 1.6330,  1.1883,  0.1030],
│                   [-0.1685,  1.1185, -0.6797],
│                   [ 1.6330,  1.1883,  0.1030],
│                   [-0.1685,  1.1185, -0.6797],
│                   [ 1.6330,  1.1883,  0.1030],
│                   [-0.1685,  1.1185, -0.6797],
│                   [ 1.6330,  1.1883,  0.1030],
│                   [-0.1685,  1.1185, -0.6797]])
└── 'x' --> <FastTreeValue 0x7f9e38f1fdc0>
    └── 'c' --> tensor([[ 0.1707,  0.0238,  0.1529,  0.9907],
                        [-1.9703,  1.0273, -0.1173, -0.6577],
                 

In [23]:
%timeit t_cat(trees)

30.1 µs ± 57.4 ns per loop (mean ± std. dev. of 7 runs, 10,000 loops each)


In [24]:
t_split = FastTreeValue.func(rise=True)(torch.split)
tree = FastTreeValue({
    'obs': torch.randn(8, 4, 84, 84),
    'action': torch.randint(0, 6, size=(8, 1,)),
    'reward': torch.rand(8, 1),
})

%timeit t_split(tree, 1)

58.7 µs ± 86.2 ns per loop (mean ± std. dev. of 7 runs, 10,000 loops each)


### Performance of Tianshou Batch

In [25]:
batches = [Batch(**_TREE_DATA_2) for _ in range(8)]

Batch.stack(batches)

Batch(
    a: tensor([[[ 1.6330,  1.1883,  0.1030],
                [-0.1685,  1.1185, -0.6797]],
       
               [[ 1.6330,  1.1883,  0.1030],
                [-0.1685,  1.1185, -0.6797]],
       
               [[ 1.6330,  1.1883,  0.1030],
                [-0.1685,  1.1185, -0.6797]],
       
               [[ 1.6330,  1.1883,  0.1030],
                [-0.1685,  1.1185, -0.6797]],
       
               [[ 1.6330,  1.1883,  0.1030],
                [-0.1685,  1.1185, -0.6797]],
       
               [[ 1.6330,  1.1883,  0.1030],
                [-0.1685,  1.1185, -0.6797]],
       
               [[ 1.6330,  1.1883,  0.1030],
                [-0.1685,  1.1185, -0.6797]],
       
               [[ 1.6330,  1.1883,  0.1030],
                [-0.1685,  1.1185, -0.6797]]]),
    x: Batch(
           c: tensor([[[ 0.1707,  0.0238,  0.1529,  0.9907],
                       [-1.9703,  1.0273, -0.1173, -0.6577],
                       [ 0.6435,  0.9898, -1.5166, -0.4396]],
         

In [26]:
%timeit Batch.stack(batches)

79.1 µs ± 143 ns per loop (mean ± std. dev. of 7 runs, 10,000 loops each)


In [27]:
Batch.cat(batches)

Batch(
    a: tensor([[ 1.6330,  1.1883,  0.1030],
               [-0.1685,  1.1185, -0.6797],
               [ 1.6330,  1.1883,  0.1030],
               [-0.1685,  1.1185, -0.6797],
               [ 1.6330,  1.1883,  0.1030],
               [-0.1685,  1.1185, -0.6797],
               [ 1.6330,  1.1883,  0.1030],
               [-0.1685,  1.1185, -0.6797],
               [ 1.6330,  1.1883,  0.1030],
               [-0.1685,  1.1185, -0.6797],
               [ 1.6330,  1.1883,  0.1030],
               [-0.1685,  1.1185, -0.6797],
               [ 1.6330,  1.1883,  0.1030],
               [-0.1685,  1.1185, -0.6797],
               [ 1.6330,  1.1883,  0.1030],
               [-0.1685,  1.1185, -0.6797]]),
    x: Batch(
           c: tensor([[ 0.1707,  0.0238,  0.1529,  0.9907],
                      [-1.9703,  1.0273, -0.1173, -0.6577],
                      [ 0.6435,  0.9898, -1.5166, -0.4396],
                      [ 0.1707,  0.0238,  0.1529,  0.9907],
                      [-1.9703,  

In [28]:
%timeit Batch.cat(batches)

144 µs ± 459 ns per loop (mean ± std. dev. of 7 runs, 10,000 loops each)


In [29]:
batch = Batch({
    'obs': torch.randn(8, 4, 84, 84),
    'action': torch.randint(0, 6, size=(8, 1,)),
    'reward': torch.rand(8, 1)}
)

%timeit list(Batch.split(batch, 1, shuffle=False, merge_last=True))

310 µs ± 3.6 µs per loop (mean ± std. dev. of 7 runs, 1,000 loops each)
